# Transferability tests (t-test for individual parameter equivalence)

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the t-test for individual parameter equivalence transferability test.

## Load packages

As usually, let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [2]:
import pandas as pd # Usual suspect #1
import numpy as np # Usual suspect #2
import matplotlib.pyplot as plt # Package that will help us plot
import seaborn as sns# Package that will help us plot a density function
from scipy.stats import qmc # we use this to generate random numbers for simulation
from scipy.stats import norm # normal distribution density function

## Load result data

### Load parameter estimates of estimation and application context

In [4]:
data_est = pd.read_csv("Task 1 & 2/car_following_model_lag1_results.csv")
data_app = pd.read_csv("car_following_model_US101_results.csv")
print(data_est)
print(data_app)

   Parameter  Estimate   s.e.  t-ratio0  p-value  Rob s.e.  Rob t-ratio0  \
0  alpha_acc     1.894  0.238     7.964    0.000     0.291         6.502   
1  alpha_dec    -2.715  0.296    -9.160    0.000     0.418        -6.499   
2   beta_acc     0.370  0.050     7.449    0.000     0.065         5.660   
3   beta_dec     0.106  0.045     2.349    0.019     0.054         1.946   
4  gamma_acc     0.737  0.055    13.318    0.000     0.074         9.944   
5  gamma_dec     0.753  0.054    14.073    0.000     0.069        10.854   
6  lamda_acc     0.926  0.035    26.232    0.000     0.036        25.388   
7  lamda_dec     0.923  0.033    28.227    0.000     0.038        24.201   
8  sigma_acc    -0.248  0.008   -30.900    0.000     0.012       -20.823   
9  sigma_dec    -0.216  0.008   -26.507    0.000     0.014       -15.858   

   Rob p-value  
0        0.000  
1        0.000  
2        0.000  
3        0.052  
4        0.000  
5        0.000  
6        0.000  
7        0.000  
8        0

#### We compute the t-test of individual parameter equivalence

In [5]:
t_test = (data_est['Estimate']-data_app['Estimate'])/np.sqrt(data_est['Rob s.e.']**2+data_app['Rob s.e.']**2)
print(t_test)

0     3.164475
1     1.075693
2     3.849695
3     2.326300
4     4.850727
5     0.672561
6     5.648279
7     4.563927
8   -39.755451
9   -34.766504
dtype: float64


We notice that for some parameters the t-test has a value > 1.96 (it can also be < -1.96). We implement a few techniques to close this gap.

## Bayesian updating

We implement Bayesian updating below

In [6]:
b_upt = ((data_est['Estimate']/(data_est['Rob s.e.']**2))+
         (data_app['Estimate']/(data_app['Rob s.e.']**2)))*((
             (1/(data_est['Rob s.e.']**2))+
                      (1/(data_app['Rob s.e.']**2))
             )**(-1))

print(b_upt)

0    1.224712
1   -2.773909
2    0.219120
3    0.027879
4    0.549721
5    0.733059
6    0.807189
7    0.831300
8    0.118492
9    0.166726
dtype: float64


## Combined transfer estimation

We implement Combined transfer estimation below 

In [7]:
a1 = data_est['Estimate']-data_app['Estimate']
a2 = data_app['Estimate']-data_est['Estimate']

b_upt = ((data_est['Estimate']/((data_est['Rob s.e.']**2)+a1*a2))+
         (data_app['Estimate']/(data_app['Rob s.e.']**2)))*((
             (1/((data_est['Rob s.e.']**2)+a1*a2))+
                      (1/(data_app['Rob s.e.']**2))
             )**(-1))

print (b_upt)

0     0.560690
1   -27.615777
2    -0.064111
3    -0.124080
4     0.026775
5     0.805792
6     0.570584
7     0.583083
8     0.373161
9     0.403196
dtype: float64


We observe that after Combined transfer estimation the data_est values get closer to the data_app values